In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp core

# ds18b20

> A module to handle DS18B20 temperature sensors

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import glob
import re
import datetime as dt
from pathlib2 import Path

In [ ]:
#export
def add_one_wire_modules():
    "Add one wire communication device kernel modules to the linux kernel"
    os.system('/usr/sbin/modprobe w1-gpio')
    os.system('/usr/sbin/modprobe w1-therm')

In [ ]:
#export
class DataPoint:
    "TODO"
    def __init__(self, device_sn, date_time, temp_raw):
        self.device_sn = device_sn
        self.date_time = date_time
        self.temp_raw = temp_raw
        
    def get_device_sn(self):
        return self.device_sn
    
    def get_date_time(self):
        return self.date_time
    
    def get_date(self):
        return self.date_time.date()
    
    def get_time(self):
        return self.date_time.time()
    
    def print_date_time(self):
        print(self.date_time.strftime('%Y-%m-%d %H:%M:%S'))
        
    def print_date(self):
        print(self.date_time.strftime('%Y-%m-%d'))
        
    def print_time(self):
        print(self.date_time.strftime('%H:%M:%S'))
        
    def get_temp_C(self):
        return self.temp_raw / 1000.0
    
    def get_temp_F(self):
        return self.get_temp_C() * 9.0 / 5.0 + 32.0
    
    def get_temp_K(self):
        return self.get_temp_C() + 273.15

In [ ]:
#export
class Device:
    "TODO"
    def __init__(self, base_dir = '/sys/bus/w1/devices/', device_sn = '28*'):
        self.base_dir = base_dir
        path = Path(base_dir).glob(device_sn + '/w1_slave')
        self.device_files = [x for x in path if x.is_file()]
        self.raw_low = None
        self.raw_high = None
        self.ref_low = None
        self.ref_high = None

    def set_calibration(self, raw_low, raw_high, ref_low, ref_high):
        self.raw_low = raw_low
        self.raw_high = raw_high
        self.ref_low = ref_low
        self.ref_high = ref_high
        self.ref_range = ref_high - ref_low
        self.raw_range = raw_high - raw_low
        
    def __read_raw(self, device_file):
        with open(device_file, 'r') as f: 
            rows = f.readlines()
            return rows
        
    def __device_sn(self, device_file):
        dev_path = str(re.findall(r'.*?/(.*?)/w1_slave', str(device_file))[0])
        return dev_path.split('/')[-1]
    
    def get_temps_raw(self):
        temps = list()
        for df in self.device_files:
            rows = self.__read_raw(df)
            while rows[0].strip()[-3:] != 'YES':
                time.sleep(0.2)
                rows = read_temp_raw()
            pos = rows[1].find('t=')
            if pos != -1:
                device_sn = self.__device_sn(df)
                date_time = dt.datetime.now()
                temperature = float(rows[1][pos + 2:])
                data_point = DataPoint(device_sn, date_time, temperature)
                temps.append(data_point)
        return temps
    
    def get_temps(self):
        temps = self.get_temps_raw()
        if self.raw_low is None:
            return temps
        for dp in temps:
            raw_value = dp.temp_raw / 1000.0
            corr_value = ((raw_value - self.raw_low) * self.ref_range / self.raw_range) + self.ref_low
            dp.temp_raw = corr_value * 1000.0
        return temps

In [ ]:
d = Device(base_dir='resources/')
d.set_calibration(-1.563, 97.75, 0.01, 98.7)

d.get_temps()[0].get_temp_C()

21.959439851781738